<a href="https://colab.research.google.com/github/swapnilsethi/Stat-5000/blob/main/RF_Model_and_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import seaborn as sns
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split #to split data into training and testing sets
import math
import sys
np.set_printoptions(threshold=sys.maxsize)  #to display all data in array
pd.pandas.set_option('display.max_columns',None) #to display all columns in dataframe
from google.colab import files #to export dataframe as a csv files 


In [95]:
# Reading feature engineered training data 
URL = ("https://raw.githubusercontent.com/swapnilsethi/Stat-5000/master/X_train.csv")
data = pd.read_csv(URL)

In [96]:
data.head()

,Unnamed: 0,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,LotFrontage_nan,MasVnrArea_nan,BsmtFinSF1_nan,BsmtFinSF2_nan,BsmtUnfSF_nan,TotalBsmtSF_nan,BsmtFullBath_nan,BsmtHalfBath_nan,GarageYrBlt_nan,GarageCars_nan,GarageArea_nan
0,0,1,60,2,65.0,8450,1,0,1,1,0,0,11,2,1,4,5,7,5,5,5,0,0,6,6,2,196.0,2,2,3,3,3,1,6,706.0,4,0.0,150.0,856.0,1,4,1,2,856,854,0,1710,1.0,0.0,2,1,3,1,2,8,3,0,0,3,5.0,2,2.0,548.0,2,3,2,0,61,0,0,0,0,0,2,2008,2,2,208500.0,0,0,0,0,0,0,0,0,0,0,0
1,1,2,20,2,80.0,9600,1,0,1,1,1,0,8,1,1,4,4,6,8,31,31,0,0,1,1,0,0.0,1,2,2,3,3,4,4,978.0,4,0.0,284.0,1262.0,1,4,1,2,1262,0,0,1262,0.0,1.0,2,0,3,1,1,6,3,1,2,3,31.0,2,2.0,460.0,2,3,2,298,0,0,0,0,0,0,5,2007,2,2,181500.0,0,0,0,0,0,0,0,0,0,0,0
2,2,3,60,2,68.0,11250,1,1,1,1,0,0,11,2,1,4,5,7,5,7,6,0,0,6,6,2,162.0,2,2,3,3,3,2,6,486.0,4,0.0,434.0,920.0,1,4,1,2,920,866,0,1786,1.0,0.0,2,1,3,1,2,6,3,1,2,3,7.0,2,2.0,608.0,2,3,2,0,42,0,0,0,0,0,9,2008,2,2,223500.0,0,0,0,0,0,0,0,0,0,0,0
3,3,4,70,2,60.0,9550,1,1,1,1,2,0,12,2,1,4,5,7,5,91,36,0,0,2,2,0,0.0,1,2,1,2,4,1,4,216.0,4,0.0,540.0,756.0,1,3,1,2,961,756,0,1717,1.0,0.0,1,0,3,1,2,7,3,1,3,1,8.0,1,3.0,642.0,2,3,2,0,35,272,0,0,0,0,2,2006,2,0,140000.0,0,0,0,0,0,0,0,0,0,0,0
4,4,5,60,2,84.0,14260,1,1,1,1,1,0,16,2,1,4,5,8,5,8,8,0,0,6,6,2,350.0,2,2,3,3,3,3,6,655.0,4,0.0,490.0,1145.0,1,4,1,2,1145,1053,0,2198,1.0,0.0,2,1,4,1,2,9,3,1,2,3,8.0,2,3.0,836.0,2,3,2,192,84,0,0,0,0,0,12,2008,2,2,250000.0,0,0,0,0,0,0,0,0,0,0,0


In [97]:
#dropping unwanted columns
nan_replacer_identification_columns = [feature for feature in data if 'nan' in feature]

data = data.drop(['Unnamed: 0'],axis=1)
data = data.drop(nan_replacer_identification_columns, axis=1)

In [98]:
X = data.drop(['SalePrice'],1)
Y = (data['SalePrice']).to_frame()

In [99]:
X = np.array(X)
Y = np.array(Y) 

Now, we will split training data into two sections. train and test to train and test model and calculate RMSE

In [100]:
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=42)

We can look at the shape of all the data to make sure we did everything correctly. We expect the training features number of columns to match the testing feature number of columns and the number of rows to match for the respective training and testing features and the labels :

In [101]:
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

Training Features Shape: (1022, 76)
Training Labels Shape: (1022, 1)
Testing Features Shape: (438, 76)
Testing Labels Shape: (438, 1)


It looks as if everything is in order!

In [102]:
y_train = y_train.flatten()
y_test =y_test.flatten()

# **Train Model**

After all the work of data preparation, creating and training the model is pretty simple using Scikit-learn. We already imported the random forest regression model from skicit-learn. 

In [103]:
# Instantiate model with 100 decision trees
base_model_regressor = RandomForestRegressor(n_estimators = 100, random_state = 42)

In [104]:
# Train the model on training data
base_model_regressor.fit(X_train, y_train)  

RandomForestRegressor(random_state=42)

 **Making Predictions on the Training Test Set and evaluate Accuracy**

Our model has now been trained to learn the relationships between the features and the targets. The next step is figuring out how good the model is! To do this we make predictions on the test features. We then compare the predictions to the known answers to calculate Accuracy.

In [105]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy
base_accuracy = evaluate(base_model_regressor, X_test, y_test)

Model Performance
Average Error: 16422.8991 degrees.
Accuracy = 90.19%.


Our average estimate is off by 16422.89 degree and Accuracy is 90.26%. That looks pretty good! Our model has learned how to predict the house sale price with 90.19% accuracy.

# **Making Predications on actual Test data**

In [106]:
# Reading test data
URL = "https://raw.githubusercontent.com/swapnilsethi/Stat-5000/master/X_test.csv"

test_data_read = pd.read_csv(URL)
# Removing unwanted columns
nan_replacer_identification_columns = [feature for feature in test_data_read if 'nan' in feature]
test_data = test_data_read.drop(['Unnamed: 0'],axis=1)
test_data = test_data.drop(nan_replacer_identification_columns, axis=1)
test_data = np.array(test_data)

# Making Predications

Y_pred = base_model_regressor.predict(test_data)

Now that we've created predictions with our model, but since this is a Kaggle competition problem, I can submit the prediction file and have it checked for correctness.

**Predication File Generator**

In [110]:
def Predications_file_generator(predications):
    ID = test_data_read.iloc[:, 1].values 
    pred = pd.DataFrame(ID, columns=['Id'])
    pred['SalePrice'] = predications.tolist()

    pred.to_csv('pred.csv')
    files.download('pred.csv')

In [ ]:
Predications_file_generator(Y_pred)

With this Model and Parameters I got RMSE of 0.14916 (Rank 2820), which I think is pretty good in a first try. 

FYI: Submissions are evaluated on Root-Mean-Squared-Error (RMSE) between the logarithm of the predicted value and the logarithm of the observed sales price. (Taking logs means that errors in predicting expensive houses and cheap houses will affect the result equally.)

# **Hyperparameter Tuning**

Though I thought I obtained really decent results with the RF model with 100 decision trees and Random test=42, I still have a lot of room to increase my rank and minimize RMSE. 

Now, in order to increase my model's performance, I'll try to find the best-suited parameters for it using the Random search method.

What is Hyperparameters? 
Hyperparmeters is like the setting of an algorithm that can be adjusted to optimize the performace.

In the case of a random forest, hyperparameters include the number of decision trees in the forest and the number of features considered by each tree when splitting a node. (The parameters of a random forest are the variables and thresholds used to split each node learned during training).

Overfitting: If we optimize the model for the training data, then our model will score very well on the training set, but will not be able to generalize to new data, such as in a test set. When a model performs highly on the training set but poorly on the test set, this is known as overfitting, or essentially creating a model that knows the training set very well but cannot be applied to new problems.


I can overcome issue of Overfitting with Cross Validation.

I am using most common K-fold cross validation technique. 

K-folds cross validation - 
In K-Fold CV, I'll split our training set into K number of subsets, called folds. Then iteratively fit the model K times, each time training the data on K-1 of the folds and evaluating on the Kth fold (called the validation data).

For example, consider fitting a model with K = 5. The first iteration I'll train on the first four folds and evaluate on the fifth. The second time I'll train on the first, second, third, and fifth fold and evaluate on the fourth. I'll repeat this procedure 3 more times, each time evaluating on a different fold. At the very end of training, I'll average the performance on each of the folds to come up with final validation metrics for the model.

**Random Search Cross Validation**

In [113]:
from pprint import pprint
# Look at parameters used by our base model
print('Parameters currently in use:\n')
pprint(base_model_regressor.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


I have used this Random forest [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) for better understanding of all these parameters.

In [131]:
#https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
#Random Hyperparameter Grid
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [100, 311, 522, 733, 944, 1155, 1366, 1577, 1788, 2000]}


In [132]:
#Random Search Training
# Use the random grid to search for best hyperparameters
# First create the base model to tune
regressor = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = regressor, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=5, random_state=42, n_jobs = 5)
# Fit the random search model
rf_random.fit(train_x, train_y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=5,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 311, 522, 733,
                                                         944, 1155, 1366, 1577,
                                                         1788, 2000]},
                   random_state=42, verbose=5)

In [133]:
rf_random.best_params_

{'bootstrap': False,
 'max_depth': None,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 311}

In [134]:
regressor = RandomForestRegressor(bootstrap= False,
 max_depth= None,
 max_features= 'sqrt',
 min_samples_leaf= 1,
 min_samples_split= 2,
 n_estimators= 311)
regressor.fit(train_x, train_y)

RandomForestRegressor(bootstrap=False, max_features='sqrt', n_estimators=311)

In [78]:
# Reading test data
URL = "https://raw.githubusercontent.com/swapnilsethi/Stat-5000/master/X_test.csv"

test_data = pd.read_csv(URL)

In [79]:
nan_replacer_identification_columns = [feature for feature in test_data if 'nan' in feature]

test_data = test_data.drop(['Unnamed: 0'],axis=1)
test_data = test_data.drop(nan_replacer_identification_columns, axis=1)

In [81]:
Y_pred = base_model_regressor.predict(test_data)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [136]:
#Evaluate Random Search
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy
base_model = RandomForestRegressor(n_estimators = 10, random_state = 42)
base_model.fit(train_x, train_y)
base_accuracy = evaluate(base_model, train_test_x, train_test_y)


Model Performance
Average Error: 18886.8245 degrees.
Accuracy = 88.38%.


In [137]:
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, train_test_x, train_test_y)


Model Performance
Average Error: 16007.3200 degrees.
Accuracy = 89.99%.


In [138]:
print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

Improvement of 1.81%.


Grid Search with Cross Validation

In [64]:
'''
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [False],
    'max_depth': [80, 90, 100, 110],
    'max_features': ['sqrt'],
    'min_samples_leaf': [1,2, 3, 4],
    'min_samples_split': [5,7,8, 10],
    'n_estimators': [100, 500, 700, 900]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = 5, verbose = 5)'''

In [ ]:
# Fit the grid search to the data
'''grid_search.fit(train_x, train_y)'''


Fitting 5 folds for each of 256 candidates, totalling 1280 fits


GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=5,
             param_grid={'bootstrap': [False], 'max_depth': [80, 90, 100, 110],
                         'max_features': ['sqrt'],
                         'min_samples_leaf': [1, 2, 3, 4],
                         'min_samples_split': [5, 7, 8, 10],
                         'n_estimators': [100, 500, 700, 900]},
             verbose=5)

In [ ]:
'''grid_search.best_params_'''

{'bootstrap': False,
 'max_depth': 100,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 700}

In [ ]:
'''best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, train_test_x, train_test_y)'''

Model Performance
Average Error: 15705.8844 degrees.
Accuracy = 90.24%.


In [ ]:
'print('Improvement of {:0.2f}%.'.format( 100 * (grid_accuracy - base_accuracy) / base_accuracy))'

Improvement of 1.86%.


In [ ]:
'''regressor = RandomForestRegressor(bootstrap= False,
 max_depth= 100,
 max_features= 'sqrt',
 min_samples_leaf= 1,
 min_samples_split= 5,
 n_estimators= 700)
regressor.fit(train_x, train_y)'''

RandomForestRegressor(bootstrap=False, max_depth=100, max_features='sqrt',
                      min_samples_split=5, n_estimators=700)

In [ ]:
'''Y_pred = regressor.predict(test_x)'''

In [ ]:
#mse = sklearn.metrics.mean_squared_error(test_y, Y_pred)

In [ ]:
#rmse = math.sqrt(mse)
#print(rmse)

In [139]:
ID = test_data.iloc[:, 1].values 


In [140]:
pred = pd.DataFrame(ID, columns=['Id'])

In [ ]:
#Y_pred_unscale = np.exp(Y_pred)

In [141]:
pred['SalePrice'] = Y_pred.tolist()

In [142]:
pred

,Id,SalePrice
0,1461,125659.556270
1,1462,150935.659164
2,1463,181814.260450
3,1464,188017.202572
4,1465,189528.681672
...,...,...
1454,2915,88181.511254
1455,2916,88246.167203
1456,2917,161140.517685
1457,2918,116839.022508


In [ ]:
from google.colab import files
pred.to_csv('pred.csv')
files.download('pred.csv')